##  Реализация web-crawler для сбора коллекции документов с указанного ресурса. 

Выбор формата хранения коллекции - .csv

### Характеристики коллекции:
- Объем собранной коллекции не менее 100 тысяч уникальных документов 
- Размер текста для каждого документа должен быть не менее 2000 символов
- Механизм очистки документов от не релевантной информации, например:
    - html теги
    - ссылки на сторонние или внутренние ресурсы сайта
    - ссылки на изображения и видео
    - знаки препинания
- Выделение признаков документа
    - автор
    - тематические теги
    - рейтинг
    - репосты
    - дата публикации 
    - другая метаинформация, предоставляемая ресурсом.

### Результатом сдачи является:
- код web-crawler
- инструмент очистки текстов
- исходная коллекция документов
- коллекция документов после очистки



## Результаты сейчас:

- Код для скачивания данных с сайта по дате
- Выгрузка осуществлена за последние 280 дней в формат html


In [ ]:
Признаки документа
 
После исследования содержимого для статьи (тег <article>)
В качестве важных признаков документа отобраны:
    
+ идентификатор статьи: <article data-story-id="7358483" >
    
+ ссылка на статью и заголовок статьи: <h2 class="story__title">
    <a class="story__title-link" 
    href="https://pikabu.ru/story/13_minut_kotoryie_ne_izmenili_mir_7356303" 
    target="_blank">13 минут, которые не изменили мир</a></h2>
</header>
    
+ идентификатор автора: ( <div class="user__info-item"><a class="user__nick story__user-link"
                         data-profile="true" href="/@AuthorName">AuthorName</a></div> )
    
+ дата публикации: <div class="user__info-item">
    <time class="caption story__datetime hint" datetime="2020-04-09T07:34:46+03:00">2 дня назад</time>
    </div>
    
+ рейтинг: <div class="story__rating-count">14812</div>
    
+- количчество просмотров: <div aria-label="619995 просмотров" class="story__views hint">
    <span class="story__views-count story__views-count_loaded">620K</span>
</div>

+ количество комментариев: <span class="story__comments-link-count">1046</span>
    
+ теги  <div class="story__tags tags">
    <a class="tags__tag" data-tag="Текст" data-tag-menu="true" 
        href="/tag/%D0%A2%D0%B5%D0%BA%D1%81%D1%82/hot" rel="noopener"
        target="_blank">Текст</a> 
    <a >Соседи</a> 
    <a >Электричество</a> 
    <a >Воровство</a>
</div>

+ текст публикации
    <div class="story__content-inner">
    И очень-очень много вложенных тегов разного вида
</div>

In [31]:
from bs4 import BeautifulSoup
import codecs
import re
import datetime
import pandas as pd
import os
import string
from tqdm import tqdm

In [32]:
def open_html(file_name):
    f = codecs.open(file_name, 'r', 'utf-8')
    document = BeautifulSoup(f.read())
    return document

def printh(text):
    print(text.prettify())

In [33]:
printh(open_html("source_hot_html/pikabu46.html"))

<html>
 <body>
  <article class="story" data-author-id="692379" data-can-vote="true" data-comments="665" data-editable="false" data-meta-rating="9211:262" data-page="false" data-rating="8949" data-slice-type="2" data-story-id="7347839" data-story-long="true" data-timestamp="1586546002" data-vid="U7347839" data-visited="false" data-vkey="sdVk" data-vote="0">
   <div class="story__left">
    <div class="story__scroll story__scroll_disabled">
     <div class="story__rating-block" data-can-vote="true" data-story-id="7347839" data-vote="0">
      <div class="story__rating-up green-is-not-red" title="Поставить плюсик">
       <svg class="icon icon--ui__rating-up icon--ui__rating-up_story" xmlns="http://www.w3.org/2000/svg">
        <use xlink:href="#icon--ui__rating-up">
        </use>
       </svg>
      </div>
      <div class="story__rating-count">
       8949
      </div>
      <div class="story__rating-down" title="Поставить минус">
       <svg class="icon icon--ui__rating-down icon--ui

In [34]:
def tags(page):
    
    try:
        text = page.find('div', {'class': 'story__tags tags'}).text
        # text = re.sub("^\s+|\n|\r|\s+$", '', text)
        text = clean(text)
        text = text.split(" ")
        # text = [str(t).lower() for t in text]
        return text
    except:
        return None

art10 = open_html("source_hot_html/pikabu46.html")
print(tags(art10))

['моё', 'аллергия', 'история', 'длиннопост', 'жалоба', 'соус', 'этикетка']


In [35]:
def clean(text):
    text = re.sub(r'^\s+|\n|\r|\t|\s+$', '', text) # очистка от отступов, переноса строки и табуляций
    # text = re.sub(r'[0-9]+', '', text) # очистка от цифр
    # возможно в дальнейшем понадобятся даты
    table = str.maketrans({key: None for key in string.punctuation}) # шаблон очистки от знаков препинания
    text = text.translate(table).lower() # очистка от знаков препинания и перевод в нижний регистр
    
    return text

In [36]:
def text(page):
    
    try:
        t = page.find('div', {'class': 'story__content-inner'}).text
        t = clean(t)
        # t = t.split(" ")
        return t
    except Exception as e:
        return None

print(text(art10))

если бы можно было бы на пару месяцев в виду карантина спрятаться от моей пищевой аллергии без продолжения её последствий я бы это сделала так как меня не раз кормили аллергеном в ресторанах кафе и это все по предварительной осведомленности об аллергии поэтому моя участь в основном  готовить себе еду самой  за время обладания  аллергией на яйца у меня уже есть четкий сценарий действий для новых блюд или неопробованных продуктов чтото новое пробуется маленьким кусочком ложечкой и требует выждать времени 15 минут если начинается реакция под рукой всегда антигистаминные глюкортикостероиды так же адреналин в ампулах спиртовые салфетки шприцы это  на случай перекрытия дыхательных путей когда 10 лет ты справляешься с такой реакцией начинаешь читать этикетки на упаковках особенно тщательно а теперь познакомьтесь с моим  последним не состоявшимся убийцей   соус mr djemius на странице товара в магазине указан состав без яиц и его производных на этикетке указан состав без яиц и его производных и

In [37]:
def amount_comments(page):
    
    try:
        t = page.find('span', {'class': 'story__comments-link-count'}).text
        return t
    except:
        return None

print(amount_comments(art10))

665


In [38]:
def rating(page):
    
    try:
        t = page.find('div', {'class': 'story__rating-count'}).text
        return t
    except:
        return None

print(rating(art10))

8949


In [39]:
def author(page):
    
    try:
        t = page.find('div', {'class': 'user__info-item'}).text
        return t
    except:
        return None

print(author(art10))

Krema


In [40]:
def title(page):
    
    try:
        t = page.find('h2', {'class': 'story__title'}).text
        t = clean(t)
        return t.lower()
    except:
        return None
    
print(title(art10))

как меня чуть не убили очередной раз


In [41]:
def link(page):
    
    try:
        t = [item['href'] for item in page.select('h2 a')]
        return t[0]
    except:
        return None

print(link(art10))

https://pikabu.ru/story/kak_menya_chut_ne_ubili_ocherednoy_raz_7347839


In [42]:
def article_id(page):
    
    try:
        t = [item['data-story-id'] for item in page.select('article')]   
        return t[0]
    except:
        return None

print(article_id(art10))

7347839


In [49]:
def views(page):
    """returns soup(page) views by users"""
    try:
        view_ = page.find("div", {"class": "story__views hint"}).text
        view_ = re.sub(r"^\s+|\n|\r|\t|\s+$", "", view_)
        view_ = view_.split(" ")
        if str(view_[0])[-1] == "K":
            view_ = float(str(view_[0])[:-1:]) * 1000
        return int(view_)
    except:
        return None
    
print(views(art10))

429000


In [44]:
def page_date(page):
    
    try:
        s = [item['datetime'] for item in page.select('time')]  
        s = str(s[0])[:10]
        s = s.split("-")
        d = datetime.date(int(s[0]), int(s[1]), int(s[2]))
        return d
    except:
        return None

print(page_date(art10))

2020-04-05


In [48]:
# полный парсинг статьи

data = []
columns = (['FileName','Title','Link','ArticleId','Date','Views','Author',
                             'Tags','AmountComments', 'Rating','Text'])

# directory = r'C:\Users\Public\Projects\python\Pikabu\InformationRetrieval\src\data\data'
directory = "source_hot_html" 
files_list = os.listdir(directory)

array = []

for f in tqdm(files_list):
    page = open_html(os.path.join(directory,f))

    array = ([f, title(page), link(page), article_id(page), page_date(page), views(page), 
             author(page), tags(page), amount_comments(page), rating(page), text(page)])
    
    data.append(dict(zip(columns, array)))
    array = []

df = pd.DataFrame(data)
df.head()

100%|██████████████████████████████████████████████████████████████████████████████| 1492/1492 [00:15<00:00, 96.36it/s]


,FileName,Title,Link,ArticleId,Date,Views,Author,Tags,AmountComments,Rating,Text
0,pikabu10.html,соседка,https://pikabu.ru/story/sosedka_7356684,7356684,2020-04-09,502000,EdwardSeed,"[моё, соседи, электричество, текст, воровство]",1205,14171,делаем с братом капитальный ремонт в квартире ...
1,pikabu100.html,карантинные будни екатеринбурга,https://pikabu.ru/story/karantinnyie_budni_eka...,7354987,2020-04-08,312000,ghazan,"[стишкипирожки, свердловск, екатеринбург, текст]",350,7018,с пустого екатеринбургасползает радужность и л...
2,pikabu1000.html,роль изоляции в учебном процессе,https://pikabu.ru/story/rol_izolyatsii_v_ucheb...,7342129,2020-04-03,85100,YuraOsinin,"[моё, дифтерия, учеба, текст, видео]",58,913,в школе я всегда был «хорошистом» иногда и тро...
3,pikabu1001.html,адаптируйся преодолевай,https://pikabu.ru/story/adaptiruysya_preodolev...,7350728,2020-04-07,79700,asdfg1975,"[собака, лестница, ветеринарный, воротник, видео]",38,892,пикабу
4,pikabu1002.html,хорошая идея,https://pikabu.ru/story/khoroshaya_ideya_7348756,7348756,2020-04-06,100000,poznikidze,"[противогаз, коронавирус, скриншот]",72,1698,httpspikaburustorynovayaoplatatruda7342610cid1...


In [46]:
df.to_csv("data_hot.csv")